# S00 — Setup & Utilities (inDrive Safety Radar)

Этот ноутбук определяет **константы, пороги, хелперы и утилиты**, которые используются во всех остальных ноутбуках.
Все зависимости подключаются осторожно: если интернета нет — блоки с установкой пропускаются.

In [ ]:
# === 1. Импорты и базовые настройки ===
import os, math, json, sys, itertools, warnings, gc
from pathlib import Path
from dataclasses import dataclass
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Опциональные пакеты (подхватываются, если установлены)
try:
    import geopandas as gpd
except Exception:
    gpd = None

try:
    import shapely
    from shapely.geometry import Point, LineString
except Exception:
    shapely = None
    Point = LineString = None

try:
    import networkx as nx
except Exception:
    nx = None

try:
    import osmnx as ox
except Exception:
    ox = None

try:
    from scipy import sparse
    from scipy.sparse.linalg import spsolve
except Exception:
    sparse = None
    spsolve = None

from typing import Tuple, List, Dict
warnings.filterwarnings("ignore")

plt.rcParams["figure.dpi"] = 150

In [ ]:
# === 2. Пути и константы проекта (новая структура) ===
from pathlib import Path

# Данные: исходные (raw) и артефакты (processed)
DATA_PATH = "data/raw/geo_locations_astana_hackathon"  # замените при необходимости
READ_DIRS = [Path("../data/processed")]
OUT_DIR = READ_DIRS[0]
OUT_DIR.mkdir(exist_ok=True, parents=True)

def resolve_path(name: str) -> Path:
    for base in READ_DIRS:
        p = base / name
        if p.exists():
            return p
    return READ_DIRS[0] / name

# Основные артефакты: читаем из существующих мест, новые файлы пишем в OUT_DIR
GRAPH_PATH = resolve_path("osm_graph.graphml")
MATCHED_PARQUET = resolve_path("matched_points.parquet")
POINT_FEATURES_PARQUET = resolve_path("features_point.parquet")
EDGE_FEATURES_PARQUET = resolve_path("features_edge.parquet")
HEX_FEATURES_PARQUET  = resolve_path("features_hex.parquet")
EVENTS_EDGE_PARQUET   = resolve_path("events_edge.parquet")
EVENTS_HEX_PARQUET    = resolve_path("events_hex.parquet")
EB_EDGE_PARQUET       = resolve_path("eb_edge.parquet")
EB_HEX_PARQUET        = resolve_path("eb_hex.parquet")
ML_EDGE_PRED_PARQUET  = resolve_path("ml_edge_pred.parquet")
SRI_EDGE_PARQUET      = resolve_path("sri_edge.parquet")
SRI_HEX_PARQUET       = resolve_path("sri_hex.parquet")
SAFE_PU_GEOJSON       = resolve_path("safe_pickups.geojson")
GEOFENCES_GEOJSON     = resolve_path("safety_geofences.geojson")

# Конфиг сохраняем в новую структуру
CONFIG = {
    "H3_RES_HEX": 10,            # ~85m
    "NEAREST_RADIUS_M": 30.0,    # радиус для микро-сегментов
    "ANGLE_TOL_DEG": 30.0,       # допуск на расхождение азимута при map-matching
    "STOP_SPEED_MS": 0.5,
    "FREEFLOW_EDGE_MS": 12.0,
    "JUNCTION_SAFE_M": 40.0,
    "HBR_THR": -3.0,             # a_long < -3 m/s^2
    "HCT_THR":  2.8,             # a_lat > 2.8 m/s^2
    "DIST2EDGE_OFFLANE_M": 9.0,
    "ALT_RESID_THR": 8.0,
    "K_ANON": 10,
    "EB_PRIORS": {
        "WWO": [1.0, 50.0],
        "ILS": [1.0, 20.0],
        "UUT": [1.0, 50.0],
        "HBR": [1.0, 40.0],
        "HCT": [1.0, 40.0],
        "GLD": [1.0, 50.0],
    },
    "SRI_WEIGHTS": {"WWO": 1.2, "ILS": 1.2, "UUT": 1.0, "HBR": 1.0, "HCT": 1.0, "GLD": 0.8, "ANOM": 0.8, "CONG": 0.6},
    "GRAPH_LAMBDA": 0.6,
}
cfg_path = OUT_DIR / "config.json"
with open(cfg_path, "w") as f:
    json.dump(CONFIG, f, indent=2)
print("Артефакты читаем из:", ', '.join(str(p) for p in READ_DIRS))
print("Артефакты сохраняем в:", OUT_DIR.resolve())

### 3. Гео‑математика: расстояния, углы, кривизна, утилиты

In [ ]:
# Быстрая евклид. аппроксимация расстояния (в метрах) для малых дистанций (Астана)
def meters_per_deg_lat(lat_deg: float) -> float:
    return 111132.92 - 559.82*math.cos(2*math.radians(lat_deg)) + 1.175*math.cos(4*math.radians(lat_deg))

def meters_per_deg_lng(lat_deg: float) -> float:
    return 111412.84*math.cos(math.radians(lat_deg)) - 93.5*math.cos(3*math.radians(lat_deg))

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2*R*math.asin(math.sqrt(a))

def bearing_deg(lat1, lon1, lat2, lon2):
    y = math.sin(math.radians(lon2-lon1))*math.cos(math.radians(lat2))
    x = math.cos(math.radians(lat1))*math.sin(math.radians(lat2)) -         math.sin(math.radians(lat1))*math.cos(math.radians(lat2))*math.cos(math.radians(lon2-lon1))
    b = math.degrees(math.atan2(y, x))
    return (b + 360.0) % 360.0

def angle_diff_deg(a, b):
    d = (a - b + 180.0) % 360.0 - 180.0
    return abs(d)

def clip_series_by_quantiles(s: pd.Series, q_lo=0.01, q_hi=0.99):
    lo, hi = s.quantile(q_lo), s.quantile(q_hi)
    return s.clip(lower=lo, upper=hi)

def eb_posterior(k, n, alpha, beta):
    # k,n могут быть массивами
    k = np.asarray(k, dtype=float)
    n = np.asarray(n, dtype=float)
    return (k + alpha) / (n + alpha + beta)

def iterative_neighbor_smoothing(values, neighbors, lam=0.6, n_iter=10):
    # Если scipy нет, простой итеративный сглаживатель: v <- (v + lam*mean(neigh))/ (1+lam)
    v = values.copy().astype(float)
    for _ in range(n_iter):
        v_new = v.copy()
        for i, neigh in neighbors.items():
            if len(neigh)==0:
                continue
            m = np.mean([v[j] for j in neigh])
            v_new[i] = (v[i] + lam*m) / (1.0 + lam)
        v = v_new
    return v

### 4. Пороги детекторов и reason‑codes

In [ ]:
REASONS = ["WWO","ILS","UUT","HBR","HCT","GLD"]
print("Reason codes:", REASONS)